In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split                
from sklearn.tree import DecisionTreeClassifier                     
from sklearn.metrics import accuracy_score                          
from sklearn.metrics import classification_report                   
from sklearn import tree   
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud , ImageColorGenerator
from PIL import Image
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import keras

import re    
import nltk  
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet

from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esgpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df = pd.read_csv('tweets.csv')
df

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0
...,...,...,...,...,...
11365,11365,wrecked,Blue State in a red sea,Media should have warned us well in advance. T...,0
11366,11366,wrecked,arohaonces,i feel directly attacked 💀 i consider moonbin ...,0
11367,11367,wrecked,🇵🇭,i feel directly attacked 💀 i consider moonbin ...,0
11368,11368,wrecked,auroraborealis,"ok who remember ""outcast"" nd the ""dora"" au?? T...",0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11370 entries, 0 to 11369
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        11370 non-null  int64 
 1   keyword   11370 non-null  object
 2   location  7952 non-null   object
 3   text      11370 non-null  object
 4   target    11370 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 444.3+ KB


In [34]:
df.isnull().sum()

id             0
keyword        0
location    3418
text           0
target         0
dtype: int64

In [35]:
print(len(pd.unique(df['keyword'])))

219


In [36]:
target_df = df.target.value_counts().reset_index()
target_df.columns = ['target', 'count']
fig = px.pie(target_df, values='count', names='target', title='Target Classification',
             color_discrete_sequence=['red', 'light blue'])
fig.show()

In [37]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))        
nltk_words = list(stopwords.words('english')) 
stop_words.extend(nltk_words)

words = [w for w in df.text if not w in stop_words]
df['words'] = words

In [38]:
import string
import re

def remove_url(words):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", words)

def remove_ablazeWord(words):
    return words.replace("ablaze", "")

def remove_punct(words):
    translator = str.maketrans("", "", string.punctuation)
    return words.translate(translator)

def remove_tripleDot(words):
    return words.replace('\u2026', "")

def remove_emojis(words):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',words)

In [39]:
df["words"] = df.words.map(remove_url)
df["words"] = df.words.map(remove_punct)
df["words"] = df.words.map(remove_tripleDot)
df["words"] = df.words.map(remove_emojis)
df["words"] = df.words.map(remove_ablazeWord)

In [40]:
import nltk
from nltk.tokenize import word_tokenize
df['words'] = [word_tokenize(entry) for entry in df['words']]
df['words'].head()

0    [Communal, violence, in, Bhainsa, Telangana, S...
1    [Telangana, Section, 144, has, been, imposed, ...
2               [Arsonist, sets, cars, at, dealership]
3               [Arsonist, sets, cars, at, dealership]
4    [Lord, Jesus, your, love, brings, freedom, and...
Name: words, dtype: object

In [41]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['j'] = wn.ADJ
tag_map['v'] = wn.VERB
tag_map['v'] = wn.ADV

In [49]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'with', 'below', 'herself', 'or', "you'll", 'too', "needn't", "aren't", 'o', 'their', 'don', 'until', 'do', 'as', "weren't", 'not', 'there', 'ma', 'against', 'how', 'mightn', 'him', 'were', 'here', "should've", 'a', 'y', 'why', 'his', 'what', 'into', 'hadn', 'ours', 've', 'between', 'off', 'had', 'ourselves', 'those', 'at', 'it', 'having', 'which', 'doing', 'some', 'didn', 'will', 'but', 'such', 'each', 'on', 'most', 'an', 'own', 'mustn', 'this', 'was', 'isn', 'about', 'shouldn', 'needn', "it's", 'than', 'couldn', 'be', 'very', 'won', 'over', 'you', 'yourselves', 'just', "hasn't", 'he', 'while', 'few', 'under', 'myself', 'these', 'can', "that'll", 'she', 'd', 'my', 'where', 'and', 'by', "isn't", 'we', "couldn't", "hadn't", "didn't", 'being', 'only', "you're", 's', 'down', 'whom', 'again', 'shan', 'the', 'me', "won't", 'yours', 'through', 'themselves', 'further', 'does', 'from', 'aren', 'during', "she's", 'then', "you'd", 'have', 'both', 'am', 'so', "haven't", 'yourself', 'if', 'is', '

In [51]:

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
for index,entry in enumerate(df['words']):
    new_words = []
    for word, tag in pos_tag(word_tokenize(text)):
        if word not in stopwords.words('english') and word.isalpha():
            word_stemmed = wordnet_lemmatizer.lemmatize(word, tag_map.get(tag[0].upper(), 'n'))
            new_words.append(word_stemmed)
    return ' '.join(new_words)

SyntaxError: 'return' outside function (1720344626.py, line 12)